In [37]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
#load data
gdf = gpd.read_file("data/cartePalaiseau.csv")
#from coordinates to polygon
for index,element in enumerate(gdf["coordinates"]):
    element=eval(element)
    coords=()
    points =Polygon([[coord[0],coord[1]] for coord in element])    
    gdf["geometry"][index]= points
    
    
#keep only the polygons columns
gdf=gdf.drop(columns=["coordinates"])
#set the crs
gdf=gdf.set_crs(crs="epsg:3857", allow_override=True)

print(gdf.crs)
gdf.head()

epsg:3857


,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((244877.665 6226270.774, 244879.456 6..."
1,21264,4A413,"POLYGON ((244952.833 6226315.319, 244953.915 6..."
2,21265,4A409,"POLYGON ((244951.228 6226305.055, 244951.825 6..."
3,21266,,"POLYGON ((244958.446 6226328.379, 244963.992 6..."
4,21267,,"POLYGON ((244964.552 6226330.920, 244968.583 6..."


# Add markers of the CartoModule to map

In [39]:
import ast
#load data
marker_gdf = gpd.read_file("data/carteModulePalaiseau.csv")
#transform the str defining the list in column coordonneesEPSG3857 into a python list
marker_gdf["coordonneesEPSG3857"] = marker_gdf["coordonneesEPSG3857"].apply(ast.literal_eval)
#transform the list of coordinates into points in geometry column
marker_gdf["geometry"] = marker_gdf["coordonneesEPSG3857"].apply(lambda coords: Point(coords))
#drop the column coordonneesEPSG3857
marker_gdf = marker_gdf.drop(columns=["coordonneesEPSG3857"])
#set the crs
marker_gdf=marker_gdf.set_crs(crs="epsg:3857", allow_override=True)
marker_gdf.head()


,macModule,idCouche,geometry
0,A8032A31204E,76,POINT (244990.444 6226329.680)
1,C45BBE39F42A,76,POINT (245025.333 6226329.187)
2,C45BBE39F9AE,76,POINT (245035.947 6226326.828)
3,A8032A311FAA,76,POINT (244983.036 6226331.224)
4,C45BBE39F56A,76,POINT (245033.645 6226333.296)


# Bind the received RSSI Data to the CartoModule Marker on Map

In [40]:
file_path="data/manip1.1.csv"
rssi_df = pd.read_csv(file_path)
rssi_df.head()
#create a different dataframe for each module with the rssi values
rssi_df = rssi_df.groupby("macModule").apply(lambda x: x.reset_index(drop=True))

#timestamp is a string, we need to convert it to datetime
rssi_df["timestamp"] = pd.to_datetime(rssi_df["timestamp"])
rssi_df.head()

timestamp     macModule  rssi
macModule                                                 
A8032A30FB9E 0 2023-06-01 12:52:26.615  A8032A30FB9E   -74
             1 2023-06-01 12:52:23.852  A8032A30FB9E   -73
             2 2023-06-01 12:52:21.148  A8032A30FB9E   -75
             3 2023-06-01 12:52:15.828  A8032A30FB9E   -78
             4 2023-06-01 12:50:20.649  A8032A30FB9E   -79

In [41]:
sampling_time=2000#ms


min_timestamp=rssi_df["timestamp"].min()+pd.Timedelta(sampling_time, unit="ms")

max_timestamp=rssi_df["timestamp"].max()
#create a list of timestamps between min_timestamp and max_timestamp with a step of sampling_time
timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()

marker_intensity_gdf_list=[]
for current_timestamp in timestamp_list:
    #for each module, we keep only the rssi values between current_timestamp and current_timestamp-sampling_time
    rssi_df_current=rssi_df[(rssi_df["timestamp"] > current_timestamp-pd.Timedelta(sampling_time, unit="ms"))&(rssi_df["timestamp"] < current_timestamp)]
    rssi_df_current=rssi_df_current.drop(columns=["timestamp","macModule"]).groupby("macModule").mean()
    marker_intensity_gdf=marker_gdf.merge(rssi_df_current, left_on="macModule", right_on="macModule")
    #mean of the rssi values for each module
    marker_intensity_gdf_list.append(marker_intensity_gdf)


marker_intensity_gdf_list

[      macModule idCouche                        geometry  rssi
 0  A8032A311FAA       76  POINT (244983.036 6226331.224) -78.0
 1  A8032A31204A       76  POINT (244987.737 6226334.275) -78.0
 2  A8032A311F6A       76  POINT (244974.011 6226332.497) -67.0
 3  A8032A311F56       76  POINT (244971.105 6226337.327) -70.0
 4  A8032A311F96       76  POINT (244979.349 6226335.888) -71.0,
       macModule idCouche                        geometry  rssi
 0  A8032A311DAA       76  POINT (244966.754 6226332.466) -66.0,
       macModule idCouche                        geometry  rssi
 0  A8032A311FAA       76  POINT (244983.036 6226331.224) -77.0
 1  A8032A311F6A       76  POINT (244974.011 6226332.497) -66.0
 2  A8032A311F56       76  POINT (244971.105 6226337.327) -70.0
 3  A8032A311F96       76  POINT (244979.349 6226335.888) -80.0,
       macModule idCouche                        geometry  rssi
 0  A8032A311F56       76  POINT (244971.105 6226337.327) -79.0,
       macModule idCouche           

In [42]:
#display the map with a heatmap of the rssi values for each module
from pyproj import Proj, transform
from pyproj import Transformer

m = folium.Map(location = [48.71242, 2.20022], tiles="Openstreetmap", zoom_start=20,crs="EPSG3857")

transformer = Transformer.from_crs("EPSG:3857","EPSG:4326")
coord_list_with_time=[]
for marker_intensity_gdf in marker_intensity_gdf_list:
    coord_list=[]
    for row in marker_intensity_gdf.iterrows():
        y,x=row[1]["geometry"].y,row[1]["geometry"].x
        longitude,latitude  = transformer.transform(x, y)
        coord_list.append([longitude,latitude,10**(-(row[1]["rssi"])/10)])
        
    coord_list_with_time.append(coord_list)
coord_list_with_time.append(coord_list)
print(coord_list_with_time[0])


[[48.713166904690254, 2.2007200581389768, 63095734.448019296], [48.71318498422266, 2.2007622901244153, 63095734.448019296], [48.71317445007082, 2.200638986440939, 5011872.336272725], [48.713203074448835, 2.200612881651723, 10000000.0], [48.71319455060883, 2.2006869316821076, 12589254.117941663]]


In [43]:
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap,HeatMapWithTime


m = folium.Map(location = [48.71242, 2.20022], tiles="Openstreetmap",max_zoom=30, zoom_start=20,crs="EPSG3857")

tooltip = folium.features.GeoJsonTooltip(fields=['ID_element', 'nom'], 
                                        labels=True,
                                        stick=False)
tooltip2 = folium.features.GeoJsonTooltip(fields=['macModule',], 
                                        labels=True,
                                        stick=False)

folium.GeoJson(gdf, tooltip=tooltip).add_to(m)
folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)
#creating the heatmapWithTime
HeatMapWithTime(coord_list_with_time,auto_play=True, radius=50).add_to(m)
folium.LayerControl().add_to(m)

m

In [44]:

m.save('data/heat_map.html')


In [45]:
test=pd.to_datetime("2023-06-01 09:56:40.412")
print(test)


2023-06-01 09:56:40.412000
